In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

### Helping Implementation

In [2]:
def AND(X):
    return all(X)

In [3]:
def split_data(df, percentage=.75):
    df = df.sample(frac=1)
    T = int(len(df)*percentage)
    return df[:T], df[T:]

In [4]:
def accuracy_score(Y,y):
    return np.sum(Y==y)/len(y)

### Generate Data

In [5]:
N,f, n, p = 1000, 2, 1, .5  # number of data instance, feature, trials, probability of each trial
X = np.random.binomial(n, p, [N,f])

In [6]:
y = np.array([int(AND(x_i)) for x_i in X]).reshape(N,1)

In [7]:
df = pd.DataFrame(data=np.hstack((X,y)), columns=['x%i'%i for i in range(f)]+['y'])

In [8]:
df.head()

,x0,x1,y
0,1,0,0
1,0,1,0
2,1,1,1
3,1,0,0
4,0,1,0


In [9]:
df_train, df_test = split_data(df)

In [10]:
X_train, y_train = df_train[df.columns[:-1]].values, df_train[df.columns[-1]].values
X_test, y_test = df_test[df.columns[:-1]].values, df_test[df.columns[-1]].values

### Algorithm

In [11]:
sigmoid = lambda x: 1/(1+np.exp(-x))

In [12]:
def weight_initialization(n_features):
    w, b = np.zeros((1,n_features)), 0
    return w,b

In [13]:
def model_optimize(w, b, X, Y):
    m = X.shape[0]
    
    # Prediction
    y_computed = sigmoid(w @ X.T + b)
    
    # Gradient calculation
    dw = (1/m)*(X.T@(y_computed - Y.T).T)
    db = (1/m)*(np.sum(y_computed - Y.T))
    
    gradient = {"dw": dw, "db": db}
    
    return gradient

In [14]:
def model_predict(w, b, X, Y, lr, iters):
    for i in range(iters):
        
        grads = model_optimize(w,b,X,Y)
        
        dw = grads["dw"]
        db = grads["db"]
        
        # Update Estimator
        w = w - lr * dw.T
        b = b - lr * db        
         
            
    coefficients = {"w": w, "b": b}
    gradient = {"dw": dw, "db": db}
    
    return coefficients, gradient

In [15]:
def predict(pred, m):
    y_pred = np.zeros((1,m))
    for i in range(pred.shape[1]):
        if pred[0][i] > 0.5:
            y_pred[0][i] = 1
    return y_pred

### Training

In [16]:
n_features = X_train.shape[1]

w, b = weight_initialization(n_features)

coefficients, gradient = model_predict(w, b, X_train, y_train, lr=0.1, iters=4500)

w = coefficients["w"]
b = coefficients["b"]

test_pred = sigmoid(w @ X_test.T + b)

### Evaluation

In [17]:
m_test =  X_test.shape[0]

test_pred = np.round(test_pred, 0)

print('Test Accuracy',accuracy_score(test_pred, y_test))

Test Accuracy 1.0
